# Аналитика по книгам (SQL)

## Введение

Перед нами представлена информация о книгах, издательствах, авторах, пользовательских обзорах книг из базы данных.

Задача: проанализировать имеющуюся информацию для дальнейшего формирования ценностного предложения для крупного сервиса по чтению книг по подписке.

В частности, планируется ответить на следующие вопросы:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг (для книги с 50 и более оценками);
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Анализ данных

Для начала создадим подключение к базе данных и выгрузим необходимые для работы таблицы.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', 
 'pwd': '_', 
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, 
 'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
query = "select * from books"
query = "select * from authors"
query = "select * from publishers"
query = "select * from ratings"
query = "select * from reviews"

In [3]:
books = pd.io.sql.read_sql(query, con = engine)
authors = pd.io.sql.read_sql(query, con = engine)
publishers = pd.io.sql.read_sql(query, con = engine)
ratings = pd.io.sql.read_sql(query, con = engine)
reviews = pd.io.sql.read_sql(query, con = engine)

Посмотрим, как выглядят наши таблицы.

In [20]:
sql = '''select * from books''' 
pd.read_sql(sql, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [21]:
pd.read_sql(sql, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [22]:
sql = '''select * from authors''' 
pd.read_sql(sql, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [23]:
pd.read_sql(sql, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [24]:
sql = '''select * from publishers''' 
pd.read_sql(sql, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [25]:
pd.read_sql(sql, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [26]:
sql = '''select * from ratings''' 
pd.read_sql(sql, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [27]:
pd.read_sql(sql, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [28]:
sql = '''select * from reviews''' 
pd.read_sql(sql, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


In [29]:
pd.read_sql(sql, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Все данные представлены в удобных форматах, таблицы легко можно объединять между собой.

Ответим на вопросы задачи.

<b> Подсчет количества книг, которые вышли после 1 января 2000 года </b>

In [14]:
sql = '''select count(book_id) as cnt from books
where publication_date >= '2000-01-01';'''
pd.read_sql(sql, con = engine)

,cnt
0,821


In [15]:
sql = '''select count(book_id) as cnt from books'''
pd.read_sql(sql, con = engine)

,cnt
0,1000


Из 1000 представленных книг 821 была выпущена в 2000 году и позже. Основная масса книг в продаже - относительные новинки.

<b> Расчет количества обзоров и средней оценки для каждой книги </b> 

In [60]:
sql = '''select books.book_id, books.title, count(review_id) as number_of_reviews, avg(subq.average_rating) as average_rating from books  
left join reviews on reviews.book_id = books.book_id
left join 
(select book_id, avg(rating) as average_rating from ratings
group by ratings.book_id) as subq
on subq.book_id = books.book_id
group by books.book_id, books.title
order by number_of_reviews desc'''
pd.read_sql(sql, con = engine)

,book_id,title,number_of_reviews,average_rating
0,948,Twilight (Twilight #1),7,3.662500
1,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
996,808,The Natural Way to Draw,0,3.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


В базе имеется информация о 1000 произведениях. Число отзывов варьируется от 0 до 7 (на книгу Twilight). Средний рейтинг также сильно различается: от 1,5 до максимальных 5 баллов. Хорошо, что никто не получил 1.

<b> Определение издательства, которое выпустило наибольшее число книг толще 50 страниц (т.е. исключая брошюры) </b>

In [17]:
sql = '''select publishers.publisher, count(distinct book_id) as number_of_books from books  
left join publishers on publishers.publisher_id = books.publisher_id
where books.num_pages > 50
group by publishers.publisher
order by number_of_books desc
limit 1'''
pd.read_sql(sql, con = engine)

,publisher,number_of_books
0,Penguin Books,42


Больше всего полноразмерных книг выпустило издательство Penguin Books - 42 штуки.

<b> Определение автора с самой высокой средней оценкой книг (по книгам с 50 и более оценками) </b>

In [67]:
sql = '''select authors.author, avg(subq.average_rating) as average_rating from books  
left join authors on authors.author_id = books.author_id
inner join (select book_id, count(rating) as number_of_ratings, avg(rating) as average_rating from ratings 
group by book_id
having count(rating) >= 50) as subq on subq.book_id = books.book_id
group by authors.author
order by average_rating desc'''
pd.read_sql(sql, con = engine)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Автор, чьи книги получили самую высокую оценку читателей - J.K. Rowling (в сотрудничестве с иллюстратором Mary GrandPré). Оценка определялась только для тех книг, которые были оценены 50 или более раз.

<b> Расчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок </b>

Воспользуемся подзапросом, чтобы по списку пользователей, поставивших более 50 оценок, и количеству написанных ими обзоров посчитать среднее.

In [19]:
sql = '''select avg(subq.number_of_reviews) as average_number_of_reviews from
(select ratings.username, count(distinct rating_id) as number_of_ratings, count(distinct review_id) as number_of_reviews from ratings  
left join reviews on reviews.username = ratings.username
group by ratings.username
having count(distinct rating_id) > 50) as subq'''
pd.read_sql(sql, con = engine)

,average_number_of_reviews
0,24.333333


Активные пользователи, поставившие более 50 оценок книгам, в среднем написали по 24 обзора.

## Выводы

По поставленным задачам можно сделать следующие выводы:
1. Из 1000 представленных книг 82% (821 штука) были выпущены в период с 1 января 2000 года.
2. Число отзывов на книги варьируется от 0 до 7. Средний рейтинг также сильно различается: от 1,5 до максимальных 5 баллов.
3. Penguin Books - издательство, которое выпустило больше всего полноразмерных книг (42 штуки).
4. Самую высокую оценку читателей получили книги J.K. Rowling в сотрудничестве с иллюстратором Mary GrandPré.
5. Среднее число обзоров на книги, которые написали пользователи, поставившие более 50 оценок, равно 24.